We now need to link together the mathematics with the provided data formats and descriptions. 

# [2] Methods

### Haplotype Reference Panel

The haplotype file, with binary encoding (provided with `-h FILENAME` argument) is a reference panel of haplotypes: 
    * Rows :: SNP at Position on given chromosome.
    * Columns :: Haplotypes
    * Data :: binary `0` or `1` indicating the base pair allele.  


Mathematically, they describe it as 
>...given a reference panel of haplotypes, $H^R=\{h_1,\ldots,h_r\}$ as input, where each haplotype is typed at $L$ biallelic sites, that is $h_i = (h_{(i,1)},\ldots,h_{(i,L)})$ and $h_{(i,j)} \in \{0, 1\}$, the \[Li and Stephens 2003, Linkage Disequilibrium model\] models each newly simulated haplotype as an imperfect mosiac of the haplotypes in $H^R$ and the haplotypes that have already been simulated (see below for more details). 

That is, $H^R$ forms a basis of vectors representing haplotypes, where each haplotype is typed at $L$ specific genetic sites.  In other words, the reference panel can be represented as (please excuse the notation): 

$$H^R = \begin{bmatrix}
h_{(1,1)} & \cdots & h_{(1,i}) & \cdots & h_{(1,r)} \\
\vdots & \ddots & \vdots & \ddots & \vdots \\
h_{(j,1)} &  & h_{(j,i)} &  & h_{(j,r)} \\
\vdots & \ddots & \vdots & \ddots & \vdots \\
h_{(L,1)} & \cdots & h_{(L,i)} & \cdots & h_{(L,r)} \\
\end{bmatrix}$$

where $r \in \mathbb{N}$ number of haplotypes are represented with the index $i \in [1,r]$, and each haplotype is typed/identified at a number $L \in \mathbb{N}$ of biallelic sites (SNPs) indexed by $j \in [1,L]$.  Then, $\forall i,j;\ h_{(i,j)} \in \{0, 1\}$.

The rows/columns were shifted in such a way to preserve their notation, but match it to the data.

### Disease SNPs

> Simulation of case-control data is based on a set of disease SNPs, $D = \{d_k:\ d_k \in \{1,\ldots,L\}\ \textrm{for}\ k=1,\ldots,K\}$...

The set of specified disease SNPs $D$ are elements of the typed biallelic sites, indexed by $k \in [1,K]$ number of total disease genetic sites.


> ...with effect sizes and $RR=\{(rr^1_k,\ rr^2_k)\}$, where $rr^1_k$ and $rr^2_k$ are the disease risks of carrying one and two copies of the \[allele in base pair at a typed site encoded as a `1`\] relative to carrying two copies of the \[allele in base pair at typed site encoded as a `0`\] at $d_k$, which combine multiplicatively across the $K$ disease SNPs. 

A tuple of relative risks $(rr^1_k, rr^2_k) \in RR$ exists for each simulated disease SNP biallelic site ($d_k \in D$).  

Here, they've stated the $rr^1$ component as the disease risk of carrying **one** copy of the `1`-encoded allele relative to carrying two copies of the `0`-encoded allele at the disease site $d_k$; whereas the $rr^2$ is the disease risk of carrying **two** copies of the `1`-encoded allele relative to carrying two copies of the `0`-encoded allele.

However, this doesn't exactly square with how we're given the specification format of Disease Loci SNPs for the program/executable.  It has:
```bash
$ ./hapgen ...[-dl POSITION ENCODED_ALLELE RR1_HET RR2_HOM]...
```
such that `POSITION` is the base-pair physical location on the chromosome for a given $d_k$ (i.e. not necessarily indexed by $k$) and the disease allele `ENCODED_ALLELE` may be specified as either `0` or `1`.  The combination of these factors indicate that `RR1_HET` is the relative risk in the heterozygous case (carrying one copy of `ENCODED_ALLELE` relative to carrying two copies of an alternate allele from the opposite base pair?), whereas `RR2_HOM` is the relative risk in the homozygous case (carry only copies of the disease allele `ENCODED_ALLELE` vs. only copies of the alternate). 

Our way to bridge between the mathematical description and the implmentation in reality consists of mutually-exclusive options. 

1. When specifying the haplotype reference panel, use `1` as a "has disease allele at location", and then make sure to specify the appropriate disease allele in the Legend and duplicate that information in all calls using the `-dl` argument (`ENCODED_ALLELE=1`).  Then, adhere to mathematical description. 

2. Make sure to adjust "on the fly" when specifying and utilizing the disease relative risk - we might have to "reverse/invert" the user-specified RR1 and RR2 to get the appropriate values for staying strictly true to the mathematical description. 

### Addtl. Haplotypes For Simulated Control and Case Individuals 

> The haplotypes $H^P = \{h_{r+1}, \ldots, h_p\}$, for the control individuals are simulated first...

For $P \in \mathbb{N}$ individuals in the "Controls" group, new haplotypes are simulated and indexed by $p \in [r+1,r+P]$.  Presumably, these are drawn from the "scaled" population and act as a sample representive (-enough) of the recombination information from HapMap. 

> ...followed by the haplotypes $H^Q=\{h_{p+1},\ldots,h_q\}$ for the case individuals. 

"Case" group of $Q \in \mathbb{N}$ simulated individuals also gets new haplotypes, and they are indexed by $q \in [r+P+1, r+P+Q]$.  

## 2.1 Simulating control data

> We simulate the control data as population controls (so that some of them may be cases) and simulate each additional haplotype $h_{i+1} \in H^P$, sequentially under the [Linkage Disequilibrium model, Li & Stephens (2003)].  

Recall that the controls group is indexed by $p$, and $p \geq r+1$ (these are not haplotypes in the reference panel input data).   

> We use the copying states $z_{(i+1, j)} \in \{1,\ldots,i\}$, which evolve in a Markov manner, to indicate the haplotype that $h_{(i+1, j)}$ copies at site $j$.

Remember, these control haplotypes are indexed by $p$, so by replacing every $i+1$ with $p$ in the above, it's more accurate to rephrase this as: 

" _The possible values of $h_{(p,j)}$, the simulated control-group individual $p$'s haplotype at a given SNP location $j$, is represented by possible "copying states" $z = z_{(p,j)}\ \in \{i\ |\ i \in [1,r+p]\}$, where the state $z$ (really, which haplotype is being copied) is determined by state transitions of a Markov chain in order to indicate the "origin" haplotype that $h_{(p,j)}$ copies._ "

#### Step 1: Simulate Cross-Over Events over input SNPs

> We simulate each haplotype in 3 stages.  First, the cross-over events, which are locations where $z_{(i+1, j)} \ne z_{(i+1, j-1))}$,...

i.e., the copying state $z_{(p,j)}$ (which haplotype is being copied) at SNP site $j$ for our new haplotype $p$ is not the same as the copying state at a previous site $z_{(p,j-1)}$.  


> ...are simulated according to the transition probabilities 
> $$P\Big({z_{(i+1, j)}} = z | z_{(i+1, j-1)}\Big) = \frac{\big(1-\mathrm{exp}(-\frac{\rho_j}{i})\big)}{i} + \mathrm{exp}\Big(- \frac{\rho_j}{i}\Big) I_z$$
> where $I_z$ is 1 if $z=z_{(i+1, j-1)}$ and 0 otherwise, and $\rho_j$ is genetic distance between SNPs $j-1$ and $j$.

Wow.  Okay, so  "cross-over events" (locations where the copying state at a given SNP site is not equal to the copy state of the previously simulated SNP) are simulated such that a state transition is evaluated using a logarithmic conditional probability.  Rephrasing the expression, 

$$P\Big(z_{(p, j)} = z | z_{(p, j-1)}\Big) = \frac{1-e^{\frac{-\rho_j}{(p-1)}}}{(p-1)} + I_z e^{\frac{-\rho_j}{(p-1)}}\\
\quad \textrm{given} \quad 
I_z = \begin{cases}
1 \quad \textrm{if } z=z_{(p, j-1)} \\
0 \quad \textrm{otherwise}
\end{cases}$$

In other words, the probability of the copying state $z_{(p,j)}$ for the simulated site $j$ being equal to some given potential transition state $z$, given the copying state $z_{(p,j-1)}$ of a previously simulated site $j-1$ on the same haplotype $p$; is equal to an exponentially decreasing value $\leq 1$ modified by a shape parameter or rate dependent on the ratio of genetic distance $\rho_j$ (between the current site and the previously simulated site) to how many haplotypes in aggregate we have simulated so far (and possibly including our reference panel?).  

This is further scaled down by the number of haplotypes simulated so far (potentially in addition to the reference panel).  

An error correction term is added back in, if indeed the possible state transition $z$ we're evaluating the probability for is the copying state of the previously simulated site $z_{(p, j-1)}$.  

In effect, this model provides the probability of a given potential state transition, as a finite discretization which incorporates error based on how many sites we are simulating.  Furthermore, the greater the genetic distance relative to the number of sites simulated so far, the more likely a particular state transition becomes.  However, a "penalty" is added for a transition to the previously used copy state, depedent in the same way on genetic distance and how many SNPs have been simulated thus far; in order to encourage maintaining the same copy state throughout the haplotype - unless the genetic distance relative to the number simulated so far is too large.  

#### Step 2: Sample Copying State for Segments between Cross-Over Events

> Conceptually, the cross-over events mimicks the effect of recombination and breaks up $h_{i+1}$ into independent segments, $\{h_{(i+1, s_1)},\ldots,h_{(i+1,s_n)}\}$, where each segment is a haplotype of SNPs between two cross-over events. 
Second, the copying state for each segment is sampled uniformly from $\{1,\ldots,i\}$. 

#### Step 3: Simulate Allele Mutation at each SNP

> Finally, the allele at each SNP is simulated condition on the copying state and a mutation parameter $\mu_i$:
$$P\Big(h_{(i+1, j)} = h_{(z,j)} | z_{(i+1, j)} = z\Big) = 1 - \mu_i$$
> _Spencer et al. (2009)_ found that $\mu_i = \frac{\theta}{2(i+\theta)}$, where $\theta = \frac{1}{\Sigma_{i=1}^m \frac{1}{n}}$, simulated amounts of novel haplotype variation similar to data simulated under the coalescent model. 

Going to need to clarify how Spencer et al. (2009) are forming and using this - no neat way to meld this with the original software implementation.  

### 2.2 Simulating case data

> We simulate the case haplotypes in a similar way, but we simulate them sequentially in pairs (with each pair corresponding to a case individual) and oversample haplotypes carrying the risk alleles based on the relative risks.

> Simulation of each haplotype pair, $(h_{i+1}, h_{i+2}) \in H^Q$, proceeds in four stages.
First, the cross-over events are simulated in the same way as for the controls, according to (1). 

> Second, the alleles at the disease SNPs are simulated.  Let $(h^1_D, h^2_D)$ be the subset of $(h_{i+1}, h_{i+2})$ that consist of the alleles at the disease SNPs, so that $h^j_D = (h_{(i+j, d_1)}, \ldots, h_{(i+j, d_k)})$ for $j=1,2$. 
> The cross-over events separate $h^1_D$ and $h^2_D$ into segments, $\{h^1_{s^1_1}, \ldots, h^1_{s^1_{n_1}}\}$ and $\{h^2_{s^2_1}, \ldots, h^2_{s^2_{n_2}}\}$

The case haplotype pairs need to be copied and split out at this stage, in the implementation.  There will be $n$ segments, which is the number of cross-over events $\pm 1$. 

> We simulate $(h^1_D, h^2_D)$ from its joint distribution, which is calculated from the relative risks and the marginal frequencies of each segment in $H^P$ and $H^R$, using Bayes Theorem:
$$\begin{aligned}
p\Big(\ (h_D^1, h_D^2)\ |\ \textrm{case}\Big) &\propto 
    p\Big( \textrm{case}\ |\ (h_D^1, h_D^2)\Big) \cdot p(h_D^1, h_D^2) \\
    &= \Pi_{k=1}^K p\Big( \textrm{case}\ |\ g_{d_k}\Big) \cdot p(h_D^1) \cdot p(h_D^2 \\
    &\propto \Big(\Pi_{k+1}^K rr_k^{g_{d_k}} \Big) \cdot \Pi_{i=1}^{n_1} p(h^1_{s^1_i}) \cdot \Pi_{j=1}^{n_2} p(h^2_{s_j^2})
\end{aligned}$$
> where $g_{d_k} = h^1_{d_k} + h^2_{d_k}$ is the genotype at $d_k$, and $p(h_s)$ is the frequency of the haplotype segment $h_s$ in $H^R$ and $H^P$. 

There's going to be some serious compute needed to do the search and comparison of segments across the reference panel and simulated controls haplotypes. 

> Third, the copying state for each segment, $h_{(i+1, s)}$, is simulated independently and is drawn uniformly from $\{1, \ldots, i\}$, like we do for the controls, if $s$ does not include any disease SNPs; or else it is drawn from 
$$P(z_{(i+1), j)} = z) \propto \Pi_{d_k \in s} \mu_{(i+1)}^{(1-I_{d_k})} \cdot (1-\mu_{i+1})^{I_{d_k}} \quad \forall j \in s$$
> where $I_{d_k}$ is 1 if $h_{(i+1, d_k)} = h_{(z, d_k)}$ and 0 otherwise.

There's going to be a lot of extra bookeeping to track whether or not a segment has a disease site in it. 

> Finally, each allele for $h_{(i+1, s)}$ is simulated according to (2). Copying states and alleles for $h_{i+2}$ are simulated in the same way. 

(2) is the allele mutation rate formula in the last step of the controls simulation data.  